# Create and Share a Mobile Map Package & Pre-planned Route

A mobile map package consolidates maps into a single file (.mmpk) that can be shared within your ArcGIS organizational account and copied directly to your mobile device. Supported layer types in the map and their data are included in the package. The mobile map package format is optimized for modern mobile devices and can run on any platform supported by ArcGIS Pro or ArcGIS Runtime. Common uses for mobile map packages include offline mapping and navigation.

Pre-planned routes are solved route layers with stops already built in. PrePlanned Routes can be shared to a worker or team on Navigator so they know exactly where they will be going, and how to get there, for the work day. 

Let's take a look at how we can automate both the creating and sharing of a **mobile map package (mmpk)** and **pre-planned route** using **ArcPy**.

## **Creating a Mobile Map Package for the United States Fire Administration (USFA)**

The USFA is sending a group to do routine inspection on the fire stations of Portland, Maine. You, as head of the team, want to have a map of all of the Portland Firestations available to you as you make your way around the city. You already have a map available on your desktop, but you'd really like to have a mobile map at the ready to access and take notes on while in the field. In this lesson you will convert your desktop map into a mobile map package. You will then share it within your organizational account so that all of your team members have access to it while you are in Portland.

## Lesson Plan

 - **Load your Project** - create a file path to the project map you wish to convert into a mmpk
 - **Prepare Your Mobile Map Package** - create your new mmpk and its output  file using the ArcPy module
 - **Share the MMPK with your USFA Team Members** - add/update your mmpk into your organization's online portal
 - **Create and share a  pre-planned route** - create a route that your team will follow while in the field
 - **Create a Navigator link** - create a link that, when clicked, will open to the preplanned route on your mobile device in Navigator for ArcGIS
 ***

 ## **Load your Project**

First you will also import the OS module for Python. This will allow you to directly connect to the databases you will be working with. 

Next you will import the ArcPy module for Python. This package will provide the data management tools necessary for this task. Once you import the ArcPy module, you will load the existing Portland Firestations Project into the environment. 

In [1]:
import os 

#establish a directory to the local drive that is storing your notebooks folder. This variable will save space later. 
notebook_dir = os.path.abspath(r"C:\Users\jos10367\Documents\GitHub\navigator-scripts\notebooks")
notebook_dir

'C:\\Users\\jos10367\\Documents\\GitHub\\navigator-scripts\\notebooks'

In [2]:
import arcpy
arcpy.env.overwriteOutput=True #clears any outputs previously made using this script 

In [7]:
aprx=arcpy.mp.ArcGISProject(os.path.join(notebook_dir, '..\data\PortlandFirestations\PortlandFirestations.aprx'))

'C:\\Users\\jos10367\\Documents\\GitHub\\navigator-scripts\\data\\PortlandFirestations\\PortlandFireStationsInspection.aprx'

You will then locate and define the map you wish to work with. You can use an ArcPy function to list all of the maps within your project - since there should only be one, you can just grab the first map that matches.

To ensure that your map was located, you can test it by calling the variable and seeing if you get a successful output.

In [8]:
firestations_map=aprx.listMaps('Fire Inspections')[0] #the zero will grab the first in the list that matches "Map"

firestations_map #testing to see if you get a successful output



## **Prepare Your Mobile Map Package**

Now that you have located the map you wish to convert into a mobile map package, you can use the ArcPy management tool: CreateMobileMapPackage. It is only required that you pass two parameters, Map file and Output File - however there are plenty of optional parameters you can add. A reference of those parameters can be found [here](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/create-mobile-map-package.htm).

Go ahead and define output_file as the complete file path name. We will use this variable here and later in the tutorial.

In [ ]:
output_file=os.path.join(notebook_dir, "..\data\PortlandFirestations\Firestations_Mobile") 

output_file=arcpy.management.CreateMobileMapPackage(in_map=firestations_map, output_file=output_file)

Allow this a few moments to complete. An output message will display with the file path for the mmpk once completed, and it should now appear in your project folder as "Firestations_Mobile.mmpk" 

## **Share the mmpk with your USFA Team Members**

Now you can share your newly created mobile map package with your team of USFA inspectors. To do so you will use the 'Share Package' tool from the data management toolset. You will connect to your GIS Portal by passing through your username and password as parameters. You are also required to add a summary of your map package and at least one tag. 

The derived output is boolean, so if the result is printed 'true' then your mmpk has been shared within your organization's portal. 

In [6]:
arcpy.SharePackage_management(output_file, <username>, <password> , "summary", "tag1, tag2")

<Result 'true'>

## **Create a Pre-Planned Route**

### **Prepare the layers that you will use in the route analysis tool**

Now you will prepare a preplanned route for your team.Preplanned routes are useful when assigning tasks involving multiple stops.For this example, you will plan a route that will take your team to every firestation that you plan to inspect for the day. 

Your team will be staying at the Embassy Suites by the Portland Airport, so you'll want to start your day there. You'll end your day with a debrief at City Hall, and of course you'll be stopping at each firestation along the way.  

The following route creation and analysis tools are documented [here](https://pro.arcgis.com/en/pro-app/tool-reference/network-analyst/an-overview-of-the-network-analyst-toolbox.htm) for reference. 


**1. Establish your workspace and network and create a route analysis layer** 

Establish a connection to the routing network you'll use to create a route analysis layer. The routing layer you will use for this exercise is called **main.Routing_ND** and is located in your PortlandFirestations folder.

Go ahead and set your workspace as your Portland Firestations geodatabase, and then connect your network to its own file path. 

In [16]:
import arcpy 

arcpy.env.workspace='C:\\Users\\jos10367\\Documents\\GitHub\\navigator-scripts\\data\\PortlandFirestations\\PortlandFirestations.gdb'
network=os.path.abspath(os.path.join(notebook_dir, "..\\data\\PortlandFirestations\\northamerica.geodatabase\\main.Routing\\main.Routing_ND"))
network

Now you can run the Make Route Analysis Layer tool.  

In [41]:
route=arcpy.na.MakeRouteAnalysisLayer(network,                    #the routing network 
                                        "RouteB",                   #the output name of your new route analysis layer
                                        "Driving Time",             #represents your mode of travel 
                                        "PRESERVE_BOTH",            #sequence - this option preserves the first and last stop 
                                        None,                       #optional time of day setting
                                        "LOCAL_TIME_AT_LOCATIONS",  #time zone setting
                                        "ALONG_NETWORK",            #controls the shape of the output lines
                                        None,                       #optional cost attributes parameter
                                        "DIRECTIONS")               #will receive turn by turn directions

**2. Import Stops**

Now you will import the stops that your team will make during their work day. They will start at their hotel, visit each firestation, and then end the day at city hall. Be sure to add the layers in the order you wish to visit them throughout the say - since we used the "PRESERVE_BOTH" parameter when making our route analysis, the first and last stops layers will be preserved. (the hotel & city hall) 

In [43]:
#connect to your local geodatabase layers

hotel=os.path.join(notebook_dir, '..\data\PortlandFirestations\PortlandFirestations.gdb\Hotel')
firestations=os.path.join(notebook_dir, '..\data\PortlandFirestations\PortlandFirestations.gdb\PortlandFireEMS')
cityhall=os.path.join(notebook_dir, '..\data\PortlandFirestations\PortlandFirestations.gdb\CityHallPortland_GeocodeAddr')

#add the layers as stops into the route analysis layer you've created 

arcpy.na.AddLocations("RouteB", "Stops", hotel) #(route analysis layer, "STOPS", stops layer)

arcpy.na.AddLocations("RouteB", "Stops", firestations)

arcpy.na.AddLocations("RouteB", "Stops", cityhall)

<Result 'RouteB'>



**3.Solve the Route**

Now that your stops are imported, you can run your route through the solve route tool. This tool will take the route layer and imported stops and generate a solved route with directions that you can then share with your team. 

In [44]:
solved_route=arcpy.na.Solve("RouteB", "SKIP", "TERMINATE", None, '')

**4. Share Route Layer with your Team**

You can now share your solved route with your team members. This will share to whatever organization you are signed in with on ArcGIS Pro. 

If you get an attribute error that reads **'ToolValidator' object has no attribute 'isLicensed'**, you can ignore it. Your route should still be shared. 

In [46]:
route_layer=arcpy.na.ShareAsRouteLayers(solved_route, route_name_prefix = "Fire Stations") #choose a recognizable name 


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

## **Create a Navigator Link**

**1. Define the item IDs of both your route layer and the map item that you will use to open the route layer in Navigator**

In [48]:
map_item='69a77ed166044909b8f7469dd4f31b84' #this is the item ID for a map of Southern Maine 

#you will need to convert your results layer to a string and then parse it to grab only its item id 
route_string = str(route_layer)
route_layer_item_id = (route_string.split("="))[1].split("'")[0]

**2. From the apps module, import the Navigator URL builder function and then create a unique URL for the route result**

 - a. create `url_type` of 'Web' that can be opened on iOS or Android devices from email, text message, etc.
 - b. set the`route_item` parameter from the route's `RouteLayerItemID` attribute
 - c. use the preferred map as the `webmap` parameter

In [49]:
from arcgis.apps import build_navigator_url 

url=build_navigator_url(url_type='Web', webmap=map_item, route_item=route_layer_item_id)
print(url)

https://navigator.arcgis.app?itemID=69a77ed166044909b8f7469dd4f31b84&routeItemID=9030631360594cb98a73e26205a5b15c


Tapping on this link will launch Navigator with the specified map and route layer.
